# MNIST DP-SGD Keras

## Imports (and Google Drive Mount in Google Colab)

In [1]:
"""Evaluate the DP-SGD optimizer using TF 1.x."""
import numpy as np
import pandas as pd
from timeit import default_timer as timer

# set tensorlfow version in google colab
try:
  %tensorflow_version 1.x
except Exception:
  pass
import tensorflow.compat.v1 as tf

# used to measure the privacy gurantee
from tensorflow_privacy.privacy.analysis.rdp_accountant import compute_rdp
from tensorflow_privacy.privacy.analysis.rdp_accountant import \
    get_privacy_spent

# optimizer used for the privacy-preserving training
from tensorflow_privacy.privacy.optimizers.dp_optimizer import \
    DPGradientDescentGaussianOptimizer

GradientDescentOptimizer = tf.train.GradientDescentOptimizer

# mount google drive
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

## Function Definitions

### Compute Privacy Budget for given Hyperparameters

In [2]:
def compute_epsilon(noise_multiplier, batch_size, target_delta,
                    trainingsset_size,
                    orders=[1 + x / 10. for x in range(1, 100)]
                    + list(range(12, 64))):
  """Computes epsilon values for given hyperparameters.

      Epsilon describes the strength of our privacy guarantee. In the case of
      DP-ML, it gives a bound on how much the probability of a particular model
      output can vary by including (or removing) a single training example. We
      usually want it to be a small constant. However, this is only an upper
      bound, and a large value of epsilon could still mean good practical
      privacy. Interpreting this value could be quiet difficult.

  Returns
  -------
  float
      Epsion-value for the expanded privacy budget.

  Parameters
  ----------
  noise_multiplier : numpy.ndarray
      Parameter to control how much noise is sampled and added to gradients
      before they are applied by the optimizer.
  batch_size : int
      Number of samples used in each training step.
  target_delta : float
      Delta bounds the probability of our privacy guarantee not holding. A rule
      of thumb is to set it to be less than the inverse of the training data
      size (i.e., the population size).
  trainingsset_size : int
      Number of samples in the trainingset.
  orders : list, optional
      List of orders, at which the Renyi divergence will be computed. If you
      are targeting a particular range of epsilons (say, 1—10) and your delta
      is fixed (say, 10^-5), then your orders must cover the range between
      1+ln(1/delta)/10=2.15 and 1+ln(1/delta)/1=12.5. The default orders are
      suitable for the mnist dataset.
  """
  # Together with the noise multiplier are these the parameters which are
  # relevant to measuring the potential privacy loss induced by the training.
  # being included in a minibatch.
  #
  # Number of steps the optimizer takes in each epoch.
  steps_per_epoch = trainingsset_size // batch_size
  #
  # The probability of an individual training point to be sampled in a
  # minibatch.
  sampling_probability = batch_size / trainingsset_size

  # List of epsilons per epoch.
  epsilon_progression = []

  rdp = 0.0

  for nm in noise_multiplier:
      if nm == 0.0:
          rdp = float('inf')
      rdp = rdp + compute_rdp(q=sampling_probability,
                              noise_multiplier=nm,
                              steps=steps_per_epoch,
                              orders=orders)
      epsilon = get_privacy_spent(orders, rdp, target_delta=target_delta)[0]
      epsilon_progression.append(epsilon)

  return epsilon_progression

### Load and Preprocess MNIST Dataset

In [3]:
def load_mnist(dataset='mnist'):
  """Loads and preprocesses the MNIST dataset.

  Returns
  -------
  tuple
      (training data, training labels, test data, test labels)
  """
  
  train, test = tf.keras.datasets.mnist.load_data()
  train_data, train_labels = train
  test_data, test_labels = test

  train_data = np.array(train_data, dtype=np.float32) / 255
  test_data = np.array(test_data, dtype=np.float32) / 255

  train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
  test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

  train_labels = np.array(train_labels, dtype=np.int32)
  test_labels = np.array(test_labels, dtype=np.int32)

  train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
  test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

  return train_data, train_labels, test_data, test_labels

### Create a simple CNN Model

In [4]:
def create_model():
  """Creates a simple example CNN.

  Returns
  -------
  tensorflow.python.keras.engine.sequential.Sequential
      A simple example CNN
  """
  from tensorflow.keras import Sequential
  from tensorflow.keras.layers import Conv2D
  from tensorflow.keras.layers import MaxPool2D
  from tensorflow.keras.layers import Flatten
  from tensorflow.keras.layers import Dense

  model = Sequential()
  model.add(Conv2D(16, 8, strides=2, padding='same', activation='relu',
                   input_shape=(28, 28, 1)))
  model.add(MaxPool2D(2, 1))
  model.add(Conv2D(32, 4, strides=2, padding='valid', activation='relu'))
  model.add(MaxPool2D(2, 1))
  model.add(Flatten())
  model.add(Dense(32, activation='relu'))
  model.add(Dense(10))

  return model

### Define and Train Model (SGD or DP-SGD alias Noisy-SGD)

In [5]:
def train_model(train_data, train_labels, test_data, test_labels, dpsgd,
                learning_rate, noise_multiplier, l2_norm_clip, batch_size,
                epochs, microbatches, callbacks=None, verbose=1):
  """Define, train and compute the used privacy budget of the keras model.

  Raises
  ------
  ValueError
      The number of microbatches must divide the batch size.

  Parameters
  ----------
  train_data : numpy.ndarray
      Array of training datapoints.
  train_labels : numpy.ndarry
      Array of labels for the training datapoints (one-hot encodding).
  test_data : numpy.ndarray
      Array of test datapoints.
  test_labels : numpy.ndarray
      Array of labels for the test datapoints (one-hot encodding).
  dpsgd : bool
      If True, train with DP-SGD. If False, train with vanilla SGD.
  learning_rate : float
      Learning rate for training.
  noise_multiplier : float
      Ratio of the standard deviation to the clipping norm. Typically more
      noise results in stronger privacy and often at the expense of utility.
  l2_norm_clip : float
      Attribute gives the maximum Euclidean norm of each individual gradient
      that is computed on an individual training example from a minibatch. This
      parameter is used to bound the optimizer's sensitivity to individual
      training points.
  batch_size : int
      Number of samples used in each training step.
  epochs : int
      Number of epochs used for the training.
  microbatches : int
      Number of microbatches (must be evently divide batch size). In practice
      clipping gradients for each exampe indivdudally can strongly degrade the
      performance because instead of parallelizing at the granularity of
      batch_size the computations must be performed for each example. Rather
      than clipping gradients per example we clip them on the basis of
      microbatches. In this way is the number of microbatches a trade-off
      parameter between privacy and utility (small number -> higher privacy,
      number closer to size of batch_size -> higher utility).
  callbacks : list, optional
      Callbacks allow to customize the behaviour of a model during training,
      evaluation and inference. Be careful, most callbacks do not work with the
      DP optimizers. Alternatively you can develop your own callbacks.
  verbose : int, optional
      Verbose parameter of the TF fit function.

  Returns
  -------
  tuple
      Training history, training time, epsilon value (see compute_epsilon())
  """
  if dpsgd and batch_size % microbatches != 0:
    raise ValueError('Number of microbatches should divide evenly size of'
                     + 'batch_size')

  if dpsgd and (len(train_data) % batch_size != 0
                or len(test_data) % batch_size != 0):
    raise ValueError('Size of minibatches should divide evenly size of'
                     + 'training- and testdatasets.')

  model = create_model()

  if dpsgd:
    optimizer = DPGradientDescentGaussianOptimizer(
        l2_norm_clip=l2_norm_clip,
        noise_multiplier=noise_multiplier,
        num_microbatches=microbatches,
        learning_rate=learning_rate)
    # Compute vector of per-example loss rather than its mean over a minibatch.
    # The optimizers needs the loss per example in order to compute the
    # gradients per example (rather than per minibatch) and clip/noise the
    # gradient of each example individually.
    loss = tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, reduction=tf.losses.Reduction.NONE)
  else:
    optimizer = GradientDescentOptimizer(learning_rate=learning_rate)
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  history = model.fit(train_data, train_labels,
                      epochs=epochs,
                      validation_data=(test_data, test_labels),
                      batch_size=batch_size,
                      verbose=verbose,
                      callbacks=callbacks)

  return model, history

### Evaluate a Model

In [6]:
def model_eval(model, x, y):
  """model.evaluate() does not work with DP-Optimizer in this simple setting.
  Use this function instead.

  Parameters
  ----------
  model : tensorflow.python.keras.engine.sequential.Sequential
      The model to be evaluated.
  x : np.ndarray
      Datapoints for the evaluation.
  y : np.ndarray
      Labels to the datapoints in x.

  Returns
  -------
  float
      Accuracy of the model on the given datapoints and labels.
  """
  correct_preds = np.sum(np.argmax(model.predict(x), axis=1)
                                    == np.argmax(y, axis=1))
  return correct_preds / len(x)

## Fit a Model

### Define Callbacks and Hyperparameter

In [7]:
# --- Callbacks

# - TF Callbacks
# * ModelCheckpoint does not work with TF-Privacy
# * TensorBoard does not work as simple callback with TF-Privacy, to get
#   TensorBoard running in TF1.x use profile_batch=0 ()

# - Custom Callbacks
import time


class RunTimeHistory(tf.keras.callbacks.Callback):
  """Callback to make runtime measuremnts.

  Attributes
  ----------
  epoch_time_start : float
      Time at the beginning of the current epoch.
  times : list
      Training time for each epoch.
  """

  def on_train_begin(self, logs={}):
      self.times = []

  def on_epoch_begin(self, batch, logs={}):
      self.epoch_time_start = time.time()

  def on_epoch_end(self, batch, logs={}):
      self.times.append(time.time() - self.epoch_time_start)


class EarlyStopping(tf.keras.callbacks.Callback):
  """Callback to stop training when the validation accuracy is at its max, i.e.
  the validation accuracy stops increasing.

  Arguments:
      patience: Number of epochs to wait after min has been hit. After this
      number of no improvement, training stops.
  """

  def __init__(self, patience=0):
    self.patience = patience

    # best_weights to store the weights at which the maximum val_acc occurs.
    self.best_weights = None

  def on_train_begin(self, logs=None):
    # The number of epoch it has waited when val_acc is not maximum
    self.wait = 0
    # The epoch the training stops at.
    self.stopped_epoch = 0
    # Initialize the best as infinity.
    self.best = 0

  def on_epoch_end(self, epoch, logs=None):
    current = logs.get('val_acc')
    if np.less(self.best, current):
      self.best = current
      self.wait = 0
      # Record the best weights if current results is better (less).
      self.best_weights = self.model.get_weights()
    else:
      self.wait += 1
      if self.wait >= self.patience:
        self.stopped_epoch = epoch
        self.model.stop_training = True

  def on_train_end(self, logs=None):
    if self.stopped_epoch > 0:
      print('Epoch %05d: early stopping' % (self.stopped_epoch + 1))
      print('Restoring model weights from the end of the best epoch.')
      self.model.set_weights(self.best_weights)


# TODO: Write custom callback for model checkpointing
# TODO: Write custom callback that stopps training after x minutes
# TODO: Write custom callback for (dynamic) learning rate decay
# TODO: Write custom callback to adjust Hyperparameter during training

# - Selected Callbacks
runtime_history = RunTimeHistory()
early_stopping = EarlyStopping(5)
callbacks = [runtime_history]

# --- Hyperparamater
dpsgd = True
learning_rate = 0.15
noise_multiplier = 1.1
l2_norm_clip = 1.0
# For the DP Optimizer the size of the minibatches must divide the number of
# training samples!
minibatches = 100
epochs = 2
# For the DP Optimizer the size of the microbatches must divide the size of the
# minibatches!
microbatches = 100
verbose=1

In [8]:
# Load training and test data.
train_data, train_labels, test_data, test_labels = load_mnist()

# Train the keras model and compute the used privacy budget.
model, history = train_model(train_data[:1000], train_labels[:1000], test_data[:1000], test_labels[:1000], dpsgd,
                             learning_rate, noise_multiplier, l2_norm_clip,
                             minibatches, epochs, microbatches, callbacks)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1000 samples, validate on 1000 samples
Epoch 1/2
1000/1000 [==============================] - 5s 5ms/sample - loss: 2.3020 - acc: 0.1110 - val_loss: 2.2799 - val_acc: 0.1730
Epoch 2/2
1000/1000 [==============================] - 3s 3ms/sample - loss: 2.2606 - acc: 0.2270 - val_loss: 2.2409 - val_acc: 0.2640


## Privacy Evaluation (only if dpsgd)

In [9]:
noise_multipliers = epochs * [noise_multiplier]
# Rule of thump: Delta is set to 1e-5 because MNIST has 60000 training
# points (see the walktrough). Delta bounds the probability that our privacy
# guarantee do not hold.
target_delta = 1e-5
# Compute the privacy budget expended
epsilon_progression = compute_epsilon(noise_multipliers, minibatches,
                                      target_delta, len(train_data))

## Collect everything potentially interesting in a dataframe
(In truth, there is a lot more interesting data produced during the training... Maybe you can find some more interesting insights yourself...)

Let's save for each epoch:
* loss
* validation loss
* accuracy
* validation accuracy
* learning rate
* noise multiplier
* l2 norm clip
* epsilon value
* runtime

In [10]:
import pandas as pd

# If we use the DP-Optimizer the history object contains the loss value for
# each minibatch in an epoch. To obtain the loss for the entire epoch, we
# calculate the average of these values.
loss = [np.mean(l) for l in history.history['loss']]
val_loss = [np.mean(l) for l in history.history['val_loss']]

learning_rates = epochs * [learning_rate]
noise_multipliers = epochs * [noise_multiplier] if dpsgd else epochs * [np.nan]
l2_norm_clips = epochs * [l2_norm_clip] if dpsgd else epochs * [np.nan]
epsilon_progression = epsilon_progression if dpsgd else epochs * [np.nan]

data = {'loss': loss,
        'val_loss': val_loss,
        'accuracy': history.history['acc'],
        'val_accuracy': history.history['val_acc'],
        'learning_rate': learning_rates,
        'noise_multiplier': noise_multipliers,
        'l2_norm_clip': l2_norm_clips,
        'epsilon': epsilon_progression,
        'runtime': runtime_history.times}

data = pd.DataFrame(data)

data

,loss,val_loss,accuracy,val_accuracy,learning_rate,noise_multiplier,l2_norm_clip,epsilon,runtime
0,2.301958,2.279926,0.111,0.173,0.15,1.1,1.0,0.843951,5.077537
1,2.260638,2.240914,0.227,0.264,0.15,1.1,1.0,0.865550,2.525313


In [11]:
# Sammel ich alle Metriken die mich interessieren?
# * Runtime Measurements
# * history: loss, acc, val_loss, val_acc
#
# Implementiere Privacy Analyse
#
# Kommentiere alles bisherige ordentlich
#
# Neue Überschirften
#
# Sammel alle Daten in einem Panda Frame
#
# Implementiere Gird-Search
#
# Implementiere Visualisierung
# minibatches => batch_size
# Update early stopping callback to use loss

## Evaluate DP-SGD or Hyperparameter-Search

### Run Experiments

In [12]:
dpsgds = [False] + 17 * [True]
learning_rates = 18 * [0.1]
noise_multipliers = [x / 10.0 for x in range(5, 16, 1)] + 6 * [1.0]
l2_norm_clips = 11 * [1.0] + [x / 10.0 for x in range(10, 16, 1)]
minibatches = 18 * [250]
microbatches = 18 * [250]
epochs = 18 * [20]
target_delta = 1e-5
runs = 3
verbose = 3

In [ ]:
data = []
dp_idx = -1
for idx, dp in enumerate(dpsgds):
  dp_idx = dp_idx + 1 if dp else dp_idx
  for run in range(runs):
    print('\n')
    print('Experiment:', idx+1, '/', len(dpsgds), 'Run: ', run+1, '/', runs)
    model, history = train_model(train_data, train_labels, test_data, test_labels, dp,
                                 learning_rates[idx], noise_multipliers[dp_idx], l2_norm_clips[dp_idx],
                                 minibatches[idx], epochs[idx], microbatches[idx], callbacks, verbose)
    
    loss = [np.mean(l) for l in history.history['loss']]
    val_loss = [np.mean(l) for l in history.history['val_loss']]
    
    learning_rates_per_epoch = epochs[idx] * [learning_rates[idx]]
    
    noise_multipliers_per_epoch = []
    l2_norm_clips_per_epoch = []
    
    if dp:
      noise_multipliers_per_epoch = epochs[idx] * [noise_multipliers[dp_idx]]
      l2_norm_clips_per_epoch = epochs[idx] * [l2_norm_clips[dp_idx]]
      epsilon_progression = compute_epsilon(noise_multipliers_per_epoch, minibatches[idx],
                                          target_delta, len(train_data))
    else:
      noise_multipliers_per_epoch = epochs[idx] * [np.nan]
      l2_norm_clips_per_epoch = epochs[idx] * [np.nan]
      epsilon_progression = epochs[idx] * [np.nan]
    
    
    run_data = ({'experiment_nr': idx,
                 'run_nr': run,
                 'loss': loss,
                 'val_loss': val_loss,
                 'accuracy': history.history['acc'],
                 'val_accuracy': history.history['val_acc'],
                 'learning_rate': learning_rates_per_epoch,
                 'noise_multiplier': noise_multipliers_per_epoch,
                 'l2_norm_clip': l2_norm_clips_per_epoch,
                 'epsilon': epsilon_progression,
                 'runtime': runtime_history.times})
    
    run_data = pd.DataFrame(run_data)
    
    data.append(run_data)
    #print(run_data.to_string())
    
    data_df = pd.concat(data)
    data_df.to_pickle('data_exp' + str(idx) + '_run' + str(run) + '.pkl')



Experiment: 1 / 18 Run:  1 / 3
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20


Experiment: 1 / 18 Run:  2 / 3
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20


Experiment: 1 / 18 Run:  3 / 3
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20


Experiment: 2 / 18 Run:  1 / 3
Train on 60000 samples, v

## Evaluate DP-SGD

In [ ]:
# We evaluate the DP-SGD with the following different
# hyperparameters in terms of epsilon, accuracy and running time. We run each
# each hyperparameter configuration (run) in a loop of 3 iterations.

dps = [False, True, True, True]
learning_rates = [0.1, 0.25, 0.15, 0.25]
noise_multipliers = [1.3, 1.1, 0.7]
clipping_thresholds = [1.5, 1, 1.5]
minibatches = 250
microbatches = 250
epochs = [20, 15, 60, 45]

# a running index
dp_idx = -1
# save the hyperparameters of each iteration in a list
rows = []
for idx, dp in enumerate(dps):
  if dp:
    dp_idx += 1

  for i in range(3):
    print('Run: ', idx, ' - Loop: ', i)

    history, training_time, epsilon = main(dp, learning_rates[idx],
                                           noise_multipliers[dp_idx],
                                           clipping_thresholds[dp_idx],
                                           minibatches, epochs[idx],
                                           microbatches, 0)
  
    accuracy = history.history['val_acc'][-1]
  
    # save the hyperparameters per row in a dict
    row = {'dp': dp,
           'learning_rate': learning_rates[idx],
           'noise_multiplier': noise_multipliers[dp_idx],
           'clipping_threshols': clipping_thresholds[dp_idx],
           'minibatches': minibatches,
           'microbatches': microbatches,
           'epochs': epochs[idx],
           'epsilon': epsilon,
           'accuracy': accuracy,
           'training_time': training_time}

    rows.append(row)

df = pd.DataFrame(rows)

In [8]:
# save/load and clean up df
#df.to_pickle('/content/drive/My Drive/df.pkl')
df = pd.read_pickle('/content/drive/My Drive/df.pkl')
df.loc[df.dp==False, ['epsilon', 'noise_multiplier', 'clipping_threshols']] = np.nan
df['Run'] = np.array([[i]*3 for i in range(4)]).flatten()
df = df.set_index('Run')
df.loc[:, 'training_time'] /= 60
df

,dp,learning_rate,noise_multiplier,clipping_threshols,minibatches,microbatches,epochs,epsilon,accuracy,training_time
Run,,,,,,,,,,
0,False,0.10,NaN,NaN,250,250,20,NaN,0.9896,0.537926
0,False,0.10,NaN,NaN,250,250,20,NaN,0.9896,0.313726
0,False,0.10,NaN,NaN,250,250,20,NaN,0.9908,0.312303
1,True,0.25,1.3,1.5,250,250,15,1.179901,0.9506,17.198169
1,True,0.25,1.3,1.5,250,250,15,1.179901,0.9473,17.125886
1,True,0.25,1.3,1.5,250,250,15,1.179901,0.9509,17.115841
2,True,0.15,1.1,1.0,250,250,60,2.969930,0.9679,68.369128
2,True,0.15,1.1,1.0,250,250,60,2.969930,0.9639,68.144310
2,True,0.15,1.1,1.0,250,250,60,2.969930,0.9656,68.915586


In [9]:
# calculate the relative standard derivation
(df.loc[:,['accuracy', 'training_time']].std(level='Run')/
df.loc[:,['accuracy', 'training_time']].mean(level='Run'))

,accuracy,training_time
Run,,
0,0.000700,0.334690
1,0.002104,0.002619
2,0.002079,0.005793
3,0.001083,0.016797


In [10]:
# calculate the mean of each run
df = df.mean(level='Run')
df

,dp,learning_rate,noise_multiplier,clipping_threshols,minibatches,microbatches,epochs,epsilon,accuracy,training_time
Run,,,,,,,,,,
0,False,0.10,NaN,NaN,250,250,20,NaN,0.990000,0.387985
1,True,0.25,1.3,1.5,250,250,15,1.179901,0.949600,17.146632
2,True,0.15,1.1,1.0,250,250,60,2.969930,0.965800,68.476342
3,True,0.25,0.7,1.5,250,250,45,7.009134,0.969533,53.398623


In [11]:
df = df.drop(labels=['minibatches', 'microbatches'], axis=1)
print(df.to_latex(index=False))

\begin{tabular}{lrrrrrrr}
\toprule
    dp &  learning\_rate &  noise\_multiplier &  clipping\_threshols &  epochs &   epsilon &  accuracy &  training\_time \\
 False &           0.10 &               NaN &                 NaN &      20 &       NaN &  0.990000 &       0.387985 \\
\midrule
  True &           0.25 &               1.3 &                 1.5 &      15 &  1.179901 &  0.949600 &      17.146632 \\
  True &           0.15 &               1.1 &                 1.0 &      60 &  2.969930 &  0.965800 &      68.476342 \\
  True &           0.25 &               0.7 &                 1.5 &      45 &  7.009134 &  0.969533 &      53.398623 \\
\bottomrule
\end{tabular}

